<a href="https://colab.research.google.com/github/AnuttaraR/DSGP_Group_03/blob/main/MediaPipe_SSLModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 44.7 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import csv
import mediapipe as mp
import numpy as np
import os
from sklearn.metrics import confusion_matrix, f1_score
from google.colab.patches import cv2_imshow


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Running Successfully

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# RoughSSLDataset - 4 people, 5 words, 5 videos each
# C:\Users\USER\Documents\DEGREE MATERIAL (Year 2)\CM2603 Data Science Group Project\RoughSSLDataaset upload this to drive
video_dir = "/content/drive/MyDrive/MediaPipe_Data/Dataset_MediaPipe"

# Initialize MediaPipe Hands model
with mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5) as hands:
    # Initialize the numpy array to hold the 3D Hand Pose data for each video
    dataset = np.zeros((100, 21, 3))

    # Loop through each video in the directory
    for i, video_file in enumerate(os.listdir(video_dir)):
        # Load the video file
        cap = cv2.VideoCapture(os.path.join(video_dir, video_file))

        # Loop through each frame of the video
        while cap.isOpened():
            # Read the next frame
            ret, image = cap.read()

            if not ret:
                break

            # Convert the image to RGB and pass it to the MediaPipe Hands model
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = hands.process(image)

            # Extract the 3D Hand Pose data from the MediaPipe Hands model output
            if results.multi_hand_landmarks:
                # Loop through each hand landmark in the list
                for hand_landmarks in results.multi_hand_landmarks:
                    # Get the landmarks for the hand
                    landmarks = hand_landmarks.landmark
                    # Convert the landmarks to a numpy array
                    landmarks_array = np.array([[landmark.x, landmark.y, landmark.z] for landmark in landmarks])
                    # Store the landmarks in the dataset array
                    dataset[i] = landmarks_array

                    # Draw the hand landmarks on the image for visualization
                    mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Show the image
            #cv2_imshow(image)
            #if cv2.waitKey(5) & 0xFF == 27:
                #break

        # Release the video capture
        cap.release()

    # Close the windows
    cv2.destroyAllWindows()

# Save the dataset to a numpy file
np.save("/content/drive/MyDrive/MediaPipe_Data/DatasetNPY_MediaPipe/dataset.npy", dataset)


In [ ]:
#Running Successfully

# Load the dataset numpy array
dataset = np.load("/content/drive/MyDrive/MediaPipe_Data/DatasetNPY_MediaPipe/dataset.npy")

# Define the path to save the CSV file
csv_path = '/content/drive/MyDrive/MediaPipe_Data/LabelsCSV_MediaPipe/labels.csv'

# Define the labels for each video in the dataset
labels = ["angry"] * 20 + ["bank"] * 20 + ["brother"] * 20 + ["bye"] * 20 + ["excuse_me"] * 20

# Write the labels to a CSV file
with open(csv_path, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["video_id", "label"])
    for i, label in enumerate(labels):
        writer.writerow([i, label])



In [ ]:
# Load dataset and labels
# Numpy array of shape (num_videos, num_frames, num_joints, 3) for 3D Hand Pose data
# and a list of labels of shape (num_videos,)
dataset = np.load("dataset.npy")
labels = pd.read_csv("labels.csv")["label"].tolist()

# Convert labels to one-hot encoding
num_classes = len(set(labels))
one_hot_labels = tf.one_hot(labels, num_classes)

# Preprocess the data
# Mean normalization and standard deviation scaling to preprocess the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
dataset = scaler.fit_transform(dataset.reshape(-1, dataset.shape[-1])).reshape(dataset.shape)

# Split the dataset into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(dataset, one_hot_labels, test_size=0.2, random_state=42)

#Using a simple convolutional neural network
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv3D(32, kernel_size=(3,3,3), activation="relu", input_shape=X_train[0].shape),
    tf.keras.layers.MaxPooling3D(pool_size=(2,2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(num_classes, activation="softmax")
])

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

#saving the model
model.save("Tf_mp_model")

# Evaluate the model on a test set
# Numpy array of shape (num_videos, num_frames, num_joints, 3) for 3D Hand Pose data and labels for each video
test_dataset = np.load("test_dataset.npy")
test_labels = pd.read_csv("test_labels.csv")["label"].tolist()
test_one_hot_labels = tf.one_hot(test_labels, num_classes)
test_dataset = scaler.transform(test_dataset.reshape(-1, test_dataset.shape[-1])).reshape(test_dataset.shape)
test_loss, test_acc = model.evaluate(test_dataset, test_one_hot_labels)

# Use the trained model to make predictions
# Numpy array of shape (num_frames, num_joints, 3) for a single video
new_data = np.load("new_data.npy")
new_data = scaler.transform(new_data.reshape(-1, new_data.shape[-1])).reshape(1, *new_data.shape)
predictions = model.predict(new_data)

In [ ]:
# Load the trained TensorFlow model
model = tf.keras.models.load_model('Tf_mp_model.h5')

# Create a MediaPipe Hands object
mp_hands = mp.solutions.hands.Hands(max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Open the camera
cap = cv2.VideoCapture(0)

while True:
    # Read a frame from the camera
    ret, frame = cap.read()

    if not ret:
        print("Failed to read frame from camera")
        break

    # Convert the frame to RGB format
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Hands to detect hand landmarks
    results = mp_hands.process(frame)

    # If hands are detected, extract the landmarks and pass them through the trained model
    if results.multi_hand_landmarks:
        # Extract the landmarks for the first hand
        hand_landmarks = results.multi_hand_landmarks[0]

        # Convert the landmarks to a numpy array
        landmarks = np.array([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark])

        # Add a batch dimension to the landmarks array
        landmarks = np.expand_dims(landmarks, axis=0)

        # Pass the landmarks through the trained model to predict the sign language word being signed
        prediction = model.predict(landmarks)
        predicted_word = np.argmax(prediction)

        # Draw the predicted word on the frame
        cv2.putText(frame, "Predicted word: {}".format(predicted_word), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Convert the frame back to BGR format for display
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    # Display the frame
    cv2.imshow("Sinhala Sign Language Recognition: ", frame)

    # Exit if the user presses the 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and destroy the display window
cap.release()
cv2.destroyAllWindows()


In [ ]:
# Load the test dataset
test_data = np.load('test_dataset.npy')
test_labels = np.loadtxt('test_labels.csv', delimiter=',')

# Load the saved model
model = tf.keras.models.load_model('Tf_mp_model')

# Evaluate the model on the test dataset
test_loss, test_acc = model.evaluate(test_data, test_labels)

# Make predictions on the test dataset
test_predictions = np.argmax(model.predict(test_data), axis=1)

# Compute the confusion matrix
cm = confusion_matrix(test_labels, test_predictions)

# Compute the F1 score
f1 = f1_score(test_labels, test_predictions, average='macro')

# Print the evaluation metrics
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)
print('Confusion Matrix:\n', cm)
print('F1 Score:', f1)